In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt
import scipy
import scipy.stats

<font size=4>
Most important word position 
    
    
Explainers agreement
</font>

In [ ]:
path = "C://Users//idobotzer//Technion//Inteligent Interactive general//Interactive Inteligence/explainers_df.csv"
explainers_df = pd.read_csv(path)

In [ ]:
explainers_df['lime_scores'] = explainers_df['lime_scores'].apply(literal_eval)
explainers_df['erase_scores'] = explainers_df['erase_scores'].apply(literal_eval)
explainers_df['saliency_scores'] = explainers_df['saliency_scores'].apply(literal_eval)

In [ ]:
def position_in_sentence(scores):
    max_index =  np.argmax(scores)
    if max_index < len(scores)/3:
        return "Beginning"
    if max_index < (len(scores)/3)*2:
        return "Middle"
    return "Ending"

In [ ]:
def positions_analysis(scores_df):
    lime_dict = {'Beginning':0, 'Middle':0, 'Ending':0}
    saliency_dict = {'Beginning':0, 'Middle':0, 'Ending':0}
    erase_dict = {'Beginning':0, 'Middle':0, 'Ending':0}   
    for index, row in explainers_df.iterrows():
        lime = row['lime_scores']
        saliency = row['saliency_scores']
        erase = row['erase_scores']
        lime_dict[position_in_sentence(lime)]+=1
        saliency_dict[position_in_sentence(saliency)]+=1
        erase_dict[position_in_sentence(erase)]+=1
    return lime_dict,saliency_dict,erase_dict

In [ ]:
lime_dict, saliency_dict, erase_dict = positions_analysis(explainers_df)
plt.bar(*zip(*lime_dict.items()))
plt.title('LIME')
plt.show()
plt.bar(*zip(*saliency_dict.items()))
plt.title('Saliency')
plt.show()
plt.bar(*zip(*erase_dict.items()))
plt.title('Erase')
plt.show()

In [ ]:
def top_k__m_matches(scores_df,k,m):
    row_count = scores_df.shape[0]
    lime_saliency_count = 0
    lime_erase_count = 0
    saliency_erase_count = 0
    for index, row in explainers_df.iterrows():
        top3_lime = extract_top_k(row['lime_scores'],k)
        top3_saliency = extract_top_k(row['saliency_scores'],k)
        top3_erase = extract_top_k(row['erase_scores'],k)
        lime_saliency_count += len(list(set(top3_lime) & set(top3_saliency)))>=m
        lime_erase_count += len(list(set(top3_lime) & set(top3_erase)))>=m
        saliency_erase_count += len(list(set(top3_saliency) & set(top3_erase)))>=m
    return lime_saliency_count/row_count, lime_erase_count/row_count, saliency_erase_count/row_count

In [ ]:
def extract_top_k(scores, k):
    sort = sorted([(x,i) for (i,x) in enumerate(scores)], reverse=True )[:k]
    return [item[1] for item in sort]

In [ ]:
def order(lst):
    s = sorted(lst, reverse=True)
    return [s.index(i) + 1 for i in lst]

In [ ]:
lime_saliency, lime_erase, saliency_erase = top_k__m_matches(explainers_df,1,1)
print(lime_saliency, lime_erase, saliency_erase)

In [ ]:
def spearman(scores_df):
    spearman_scores = []
    lime_saliency_count = 0
    lime_erase_count = 0
    saliency_erase_count = 0
    for index, row in explainers_df.iterrows():
        lime = list(row['lime_scores'])
        saliency = list(row['saliency_scores'])
        erase = list(row['erase_scores'])
        spearman_scores.append([scipy.stats.spearmanr(lime,saliency)[0],scipy.stats.spearmanr(lime,erase)[0], scipy.stats.spearmanr(saliency,erase)[0]])
    return spearman_scores

In [ ]:
spearman_scores=spearman(explainers_df)

In [ ]:
lime_saliency=[item[0] for item in spearman_scores]
print(sum(lime_saliency)/len(lime_saliency))

In [ ]:
lime_erase=[item[1] for item in spearman_scores]
print(sum(lime_erase)/len(lime_erase))

In [ ]:
saleincy_erase=[item[2] for item in spearman_scores]
print(sum(saleincy_erase)/len(saleincy_erase))

In [ ]:
corr_dict = {0:0, 1:0, 2:0}
for i,sentence in enumerate(spearman_scores):
    max_index =  np.argmax(sentence)
    corr_dict[max_index]+=1/500

In [ ]:
corr_dict